# Modules

In [1]:
import tensorflow as tf
import scipy
from tensorflow.python.client import device_lib
from tensorflow.keras.applications import InceptionV3, ResNet152V2
from keras.models import Sequential 
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dropout, Input, Flatten, Dense, MaxPooling2D, BatchNormalization,AveragePooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [3]:
import tensorflow as tf
print(tf.__version__)

2.10.1


In [4]:
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [5]:
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 10538972037154707131
 xla_global_id: -1,
 name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 2240439911
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 5019137706932479803
 physical_device_desc: "device: 0, name: NVIDIA GeForce GTX 1650, pci bus id: 0000:07:00.0, compute capability: 7.5"
 xla_global_id: 416903419]

# Dataset preparation

In [6]:
batchsize = 8

In [7]:
# Training dataset

train_data_gen = ImageDataGenerator(rotation_range=0.2, 
                                        shear_range=0.2,
                                        zoom_range=0.2, 
                                        width_shift_range=0.2,
                                        height_shift_range=0.2, 
                                        rescale=1./255,
                                        validation_split=0.2)

train_data = train_data_gen.flow_from_directory('dataset/train', 
                                                    target_size=(80, 80), 
                                                    batch_size=batchsize, 
                                                    class_mode='categorical',
                                                    subset='training'
                                                    )

validation_data = train_data_gen.flow_from_directory('dataset/train', 
                                                    target_size=(80, 80), 
                                                    batch_size=batchsize, 
                                                    class_mode='categorical',
                                                    subset='validation'
                                                    )

Found 64719 images belonging to 2 classes.
Found 16179 images belonging to 2 classes.


In [8]:
# Testing dataset

test_data_gen = ImageDataGenerator(rescale=1./255)
test_data = test_data_gen.flow_from_directory('dataset/test', 
                                                    target_size=(80, 80), 
                                                    batch_size=batchsize, 
                                                    class_mode='categorical'
                                                    )

Found 4000 images belonging to 2 classes.


# Model preparation

In [9]:
# Get knowledge from InceptionV3 pretrained model, so we don't need to train the model from scratch
bmodel = InceptionV3(include_top=False, weights='imagenet', input_tensor=Input(shape=(80, 80, 3)))
hmodel = bmodel.output
hmodel = Flatten()(hmodel)

# The only layers we need to train
# This is the layers' illustration https://www.google.com/url?sa=i&url=https%3A%2F%2Ftowardsdatascience.com%2Fthe-most-intuitive-and-easiest-guide-for-convolutional-neural-network-3607be47480&psig=AOvVaw0CYHQ2oruhWMmKK2Lxm5cN&ust=1668996857968000&source=images&cd=vfe&ved=0CBAQjRxqFwoTCLDJs9TYu_sCFQAAAAAdAAAAABAU
hmodel = Dense(64, activation='relu')(hmodel)
hmodel = Dropout(0.5)(hmodel)
hmodel = Dense(2, activation='softmax')(hmodel)

model = Model(inputs=bmodel.input, outputs=hmodel)

for layer in bmodel.layers:
    layer.trainable = False

print('done')

done


In [241]:
# # Get knowledge from InceptionV3 pretrained model, so we don't need to train the model from scratch
# bmodel = ResNet152V2(include_top=False, weights='imagenet', input_tensor=Input(shape=(80, 80, 3)))
# hmodel = bmodel.output
# hmodel = AveragePooling2D(pool_size = (3,3))(hmodel)
# hmodel = Flatten(name= 'flatten')(hmodel)

# # The only layers we need to train
# # This is the layers' illustration https://www.google.com/url?sa=i&url=https%3A%2F%2Ftowardsdatascience.com%2Fthe-most-intuitive-and-easiest-guide-for-convolutional-neural-network-3607be47480&psig=AOvVaw0CYHQ2oruhWMmKK2Lxm5cN&ust=1668996857968000&source=images&cd=vfe&ved=0CBAQjRxqFwoTCLDJs9TYu_sCFQAAAAAdAAAAABAU

# hmodel = Dense(256, activation = "relu")(hmodel)
# hmodel = Dropout(0.25)(hmodel)
# hmodel = Dense(256, activation = "relu")(hmodel)
# hmodel = Dropout(0.25)(hmodel)
# hmodel = Dense(256, activation = "relu")(hmodel)
# hmodel = Dropout(0.25)(hmodel)
# hmodel = Dense(2, activation='softmax')(hmodel)

# model = Model(inputs=bmodel.input, outputs=hmodel)


# for layer in bmodel.layers:
#     layer.trainable = False

# print('done')

done


In [10]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 80, 80, 3)]  0           []                               
                                                                                                  
 conv2d (Conv2D)                (None, 39, 39, 32)   864         ['input_1[0][0]']                
                                                                                                  
 batch_normalization (BatchNorm  (None, 39, 39, 32)  96          ['conv2d[0][0]']                 
 alization)                                                                                       
                                                                                                  
 activation (Activation)        (None, 39, 39, 32)   0           ['batch_normalization[0][0]']

 ormalization)                                                                                    
                                                                                                  
 batch_normalization_11 (BatchN  (None, 7, 7, 32)    96          ['conv2d_11[0][0]']              
 ormalization)                                                                                    
                                                                                                  
 activation_5 (Activation)      (None, 7, 7, 64)     0           ['batch_normalization_5[0][0]']  
                                                                                                  
 activation_7 (Activation)      (None, 7, 7, 64)     0           ['batch_normalization_7[0][0]']  
                                                                                                  
 activation_10 (Activation)     (None, 7, 7, 96)     0           ['batch_normalization_10[0][0]'] 
          

                                                                                                  
 conv2d_23 (Conv2D)             (None, 7, 7, 96)     55296       ['activation_22[0][0]']          
                                                                                                  
 batch_normalization_20 (BatchN  (None, 7, 7, 48)    144         ['conv2d_20[0][0]']              
 ormalization)                                                                                    
                                                                                                  
 batch_normalization_23 (BatchN  (None, 7, 7, 96)    288         ['conv2d_23[0][0]']              
 ormalization)                                                                                    
                                                                                                  
 activation_20 (Activation)     (None, 7, 7, 48)     0           ['batch_normalization_20[0][0]'] 
          

 conv2d_34 (Conv2D)             (None, 3, 3, 128)    98304       ['mixed3[0][0]']                 
                                                                                                  
 batch_normalization_34 (BatchN  (None, 3, 3, 128)   384         ['conv2d_34[0][0]']              
 ormalization)                                                                                    
                                                                                                  
 activation_34 (Activation)     (None, 3, 3, 128)    0           ['batch_normalization_34[0][0]'] 
                                                                                                  
 conv2d_35 (Conv2D)             (None, 3, 3, 128)    114688      ['activation_34[0][0]']          
                                                                                                  
 batch_normalization_35 (BatchN  (None, 3, 3, 128)   384         ['conv2d_35[0][0]']              
 ormalizat

 activation_44 (Activation)     (None, 3, 3, 160)    0           ['batch_normalization_44[0][0]'] 
                                                                                                  
 conv2d_45 (Conv2D)             (None, 3, 3, 160)    179200      ['activation_44[0][0]']          
                                                                                                  
 batch_normalization_45 (BatchN  (None, 3, 3, 160)   480         ['conv2d_45[0][0]']              
 ormalization)                                                                                    
                                                                                                  
 activation_45 (Activation)     (None, 3, 3, 160)    0           ['batch_normalization_45[0][0]'] 
                                                                                                  
 conv2d_41 (Conv2D)             (None, 3, 3, 160)    122880      ['mixed4[0][0]']                 
          

 ormalization)                                                                                    
                                                                                                  
 activation_55 (Activation)     (None, 3, 3, 160)    0           ['batch_normalization_55[0][0]'] 
                                                                                                  
 conv2d_51 (Conv2D)             (None, 3, 3, 160)    122880      ['mixed5[0][0]']                 
                                                                                                  
 conv2d_56 (Conv2D)             (None, 3, 3, 160)    179200      ['activation_55[0][0]']          
                                                                                                  
 batch_normalization_51 (BatchN  (None, 3, 3, 160)   480         ['conv2d_51[0][0]']              
 ormalization)                                                                                    
          

                                                                                                  
 conv2d_66 (Conv2D)             (None, 3, 3, 192)    258048      ['activation_65[0][0]']          
                                                                                                  
 batch_normalization_61 (BatchN  (None, 3, 3, 192)   576         ['conv2d_61[0][0]']              
 ormalization)                                                                                    
                                                                                                  
 batch_normalization_66 (BatchN  (None, 3, 3, 192)   576         ['conv2d_66[0][0]']              
 ormalization)                                                                                    
                                                                                                  
 activation_61 (Activation)     (None, 3, 3, 192)    0           ['batch_normalization_61[0][0]'] 
          

                                                                                                  
 batch_normalization_74 (BatchN  (None, 3, 3, 192)   576         ['conv2d_74[0][0]']              
 ormalization)                                                                                    
                                                                                                  
 activation_70 (Activation)     (None, 3, 3, 192)    0           ['batch_normalization_70[0][0]'] 
                                                                                                  
 activation_74 (Activation)     (None, 3, 3, 192)    0           ['batch_normalization_74[0][0]'] 
                                                                                                  
 conv2d_71 (Conv2D)             (None, 1, 1, 320)    552960      ['activation_70[0][0]']          
                                                                                                  
 conv2d_75

 activation_82 (Activation)     (None, 1, 1, 384)    0           ['batch_normalization_82[0][0]'] 
                                                                                                  
 activation_83 (Activation)     (None, 1, 1, 384)    0           ['batch_normalization_83[0][0]'] 
                                                                                                  
 batch_normalization_84 (BatchN  (None, 1, 1, 192)   576         ['conv2d_84[0][0]']              
 ormalization)                                                                                    
                                                                                                  
 activation_76 (Activation)     (None, 1, 1, 320)    0           ['batch_normalization_76[0][0]'] 
                                                                                                  
 mixed9_0 (Concatenate)         (None, 1, 1, 768)    0           ['activation_78[0][0]',          
          

                                                                                                  
 activation_85 (Activation)     (None, 1, 1, 320)    0           ['batch_normalization_85[0][0]'] 
                                                                                                  
 mixed9_1 (Concatenate)         (None, 1, 1, 768)    0           ['activation_87[0][0]',          
                                                                  'activation_88[0][0]']          
                                                                                                  
 concatenate_1 (Concatenate)    (None, 1, 1, 768)    0           ['activation_91[0][0]',          
                                                                  'activation_92[0][0]']          
                                                                                                  
 activation_93 (Activation)     (None, 1, 1, 192)    0           ['batch_normalization_93[0][0]'] 
          

# Train the model

In [11]:
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

In [12]:
checkpoint = ModelCheckpoint('models', 
                             monitor='val_loss', 
                             save_best_only=True, 
                             verbose=3)

# patience = max num of epochs when accuracy is not increasing
earlystop = EarlyStopping(monitor='val_loss', patience=7, verbose=3, restore_best_weights=True)

learning_rate = ReduceLROnPlateau(monitor='val_loss', patience=3, verbose=3)

callbacks = [checkpoint, earlystop, learning_rate]

In [13]:
try:
    model.compile(optimizer='RMSprop', loss='categorical_crossentropy', metrics=['accuracy'])

    model.fit(train_data,
          steps_per_epoch=train_data.samples//batchsize,
          validation_data=validation_data,
          validation_steps=validation_data.samples//batchsize,
          callbacks=callbacks,
          epochs=50)
except Exception as e:
    print(e)

Epoch 1/50
8088/8089 [============================>.] - ETA: 0s - loss: 0.2507 - accuracy: 0.9176
Epoch 1: val_loss improved from inf to 0.19872, saving model to models


INFO:tensorflow:Assets written to: models\assets


INFO:tensorflow:Assets written to: models\assets


8089/8089 [==============================] - 1100s 135ms/step - loss: 0.2507 - accuracy: 0.9176 - val_loss: 0.1987 - val_accuracy: 0.9239 - lr: 0.0010
Epoch 2/50
8088/8089 [============================>.] - ETA: 0s - loss: 0.2303 - accuracy: 0.9263
Epoch 2: val_loss did not improve from 0.19872
8089/8089 [==============================] - 616s 76ms/step - loss: 0.2304 - accuracy: 0.9263 - val_loss: 0.2057 - val_accuracy: 0.9243 - lr: 0.0010
Epoch 3/50
8088/8089 [============================>.] - ETA: 0s - loss: 0.2306 - accuracy: 0.9252
Epoch 3: val_loss did not improve from 0.19872
8089/8089 [==============================] - 416s 51ms/step - loss: 0.2306 - accuracy: 0.9252 - val_loss: 0.2007 - val_accuracy: 0.9275 - lr: 0.0010
Epoch 4/50
8089/8089 [==============================] - ETA: 0s - loss: 0.2282 - accuracy: 0.9290
Epoch 4: val_loss did not improve from 0.19872

Epoch 4: ReduceLROnPlateau reducing learning rate to 0.00010000000474974513.
8089/8089 [===========================

# Model evaluation

In [14]:
acc_training, loss_training = model.evaluate(train_data)
print(acc_training)
print(loss_training)

8090/8090 [==============================] - 267s 33ms/step - loss: 0.1922 - accuracy: 0.9311
0.19217592477798462
0.9311021566390991


In [16]:
acc_val, loss_val = model.evaluate(validation_data)
print(acc_val)
print(loss_val)

2023/2023 [==============================] - 79s 39ms/step - loss: 0.1976 - accuracy: 0.9206
0.19758740067481995
0.920637845993042


In [17]:
acc_test, loss_test = model.evaluate(test_data)
print(acc_test)
print(loss_test)

500/500 [==============================] - 16s 31ms/step - loss: 0.4668 - accuracy: 0.8310
0.4667949080467224
0.8309999704360962
